In [10]:
!pip install transformers 
!pip install accelerate 
!pip install bitsandbytes
!pip install peft
!pip install datasets

In [13]:
import pandas as pd
import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer, 
    AutoModelForCausalLM, 
    BitsAndBytesConfig, 
    DataCollatorForLanguageModeling, 
    Trainer, 
    TrainingArguments
)
from peft import LoraConfig, get_peft_model, PeftModel

from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("wanb-api-key")


# step 1: using llama 3.2 
base_model = "/kaggle/input/llama-3.2/transformers/1b/1"

df = pd.read_csv('/kaggle/input/questions/questions.csv')
df = df.iloc[:, :2]

def format_example(row):
    return f"Question: {row['Question']}\nAnswer: {row['Answer']}"

df['text'] = df.apply(format_example, axis=1)
dataset = Dataset.from_pandas(df[['text']])

# step 2: loading the tokenizer for llama 3.2
tokenizer = AutoTokenizer.from_pretrained(base_model, use_fast=False)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# step 3: quantization config using bitsandbytes
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True
)

# step 4: loading llama 3.2 model 
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    return_dict=True,
    low_cpu_mem_usage=True,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config
)

model.config.pad_token_id = tokenizer.pad_token_id

# tokenzing dataset
def tokenize_function(examples):
    return tokenizer(
        examples['text'],
        padding='max_length',
        truncation=True,
        max_length=512,
        return_tensors='pt'
    )

tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=['text'])
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])

# step 5: lora config
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "down_proj", "up_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

# step 6: training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=10,  # reduced from 100 because of gpu constraints
    per_device_train_batch_size=8,  
    gradient_accumulation_steps=2, 
    learning_rate=5e-4,  
    fp16=True,  
    logging_steps=10,
    save_steps=100,  
    save_total_limit=2,
    evaluation_strategy='no',
    optim="paged_adamw_8bit",
    ddp_find_unused_parameters=False
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

# step 7: train 
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator
)

trainer.train()

# step 8: save the peft model
model.save_pretrained('./finetuned_llama')
tokenizer.save_pretrained('./finetuned_llama')

# step 9: load the finetuned model and generate answers
base_model_instance = AutoModelForCausalLM.from_pretrained(
    base_model,
    device_map='auto',
    trust_remote_code=True,
    quantization_config=bnb_config,
    torch_dtype=torch.float16
)

# loading the tokenizer and lora-adapted model
tokenizer = AutoTokenizer.from_pretrained('./finetuned_llama', use_fast=False)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = PeftModel.from_pretrained(base_model_instance, './finetuned_llama')
model.config.pad_token_id = tokenizer.pad_token_id

def generate_answer(question, max_length=200):
    input_text = f"Question: {question}\nAnswer:"
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to('cuda')
    output = model.generate(
        input_ids,
        max_length=max_length,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        early_stopping=True,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id
    )
    answer = tokenizer.decode(output[0], skip_special_tokens=True)
    # Extract the answer part
    if "Answer:" in answer:
        answer = answer.split("Answer:")[1].strip()
    return answer

# trying an example 
question = "How can I implement obstacle avoidance in ROS2 using Nav2?"
answer = generate_answer(question)
print("Question:", question)
print("Answer:", answer)


Map:   0%|          | 0/3258 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss
10,2.345100
20,2.089700
30,1.961400
40,1.993300
50,1.950600
60,1.927000
70,1.818000
80,1.863100
90,1.947700
100,1.826500


/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:638: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Question: How can I implement obstacle avoidance in ROS2 using Nav2?
Answer: "Nav2 provides distance and velocity sensors for obstacle detection. I would recommend integrating these sensors with ROS 2's obstacle avoiding module and adding physical properties like the size of the obstacle." This can be done by configuring the robot state representation and integrating the distance sensor data. Then, you can avoid obstacles using the same algorithm as inROS 1."


In [14]:
question = "Tell me how can I navigate to a specific pose - include replanning aspects in your answer."
answer = generate_answer(question)
print("Question:", question)
print("Answer:", answer)


Question: Tell me how can I navigate to a specific pose - include replanning aspects in your answer.
Answer: To navigate specifically toa pose, you can use the command: ros2 pose --ros-args --params-file params.yaml --node-name my_node /pose:=/pose'". This will direct the robot to follow a particular pose." For more detailed navigation instructions, refer to the navigation2 example and parameters.


In [18]:
question = "Tell me how can I navigate to a specific pose - include replanning aspects in your answer. Can you provide me with code for this task?"
answer = generate_answer(question)
print("Question:", question)
print("Answer:", answer)

Question: Tell me how can I navigate to a specific pose - include replanning aspects in your answer. Can you provide me with code for this task?
Answer: Yes, you can use the 'ros2 pose' tool to set a goal pose and manage replanation details. For example, use 'ROS2 Pose --goal 2 3 0' to adjust a target pose. Additionally, check out the demo from the ROS 2024 ROS Core repository for detailed guidance on navigating to targets. It includes both pure ROS and Gazebo-based solutions. Finally, if you want to plan a route to reach a pose, consider using the "ros3 route" tool. These tools make it easier to manage complex movement tasks in ROS. You can find more examples and detailed instructions in the repository's documentation. I hope this helps with navigation inROS. Let me know if I missed any crucial features. Thank you for including me in this community!


In [22]:
# # pushing to hf
import os
from huggingface_hub import HfApi

os.environ['hugging_face_token'] = ' '
api = HfApi(token=os.environ['hugging_face_token'])

user_info = api.whoami()
user = user_info['name']
print(f"Logged in as: {user}")

repo_name = "finetuned-llama"
repo_id = f"{user}/{repo_name}"

api.create_repo(repo_id=repo_id, private=False, exist_ok=True)

model.push_to_hub(repo_id, token=os.environ['hugging_face_token'])
tokenizer.push_to_hub(repo_id, token=os.environ['hugging_face_token'])


Logged in as: krishmurjani


adapter_model.safetensors:   0%|          | 0.00/45.1M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/krishmurjani/finetuned-llama/commit/3a17b409a725a1026bcae8c5aad0f239d98d3b7f', commit_message='Upload tokenizer', commit_description='', oid='3a17b409a725a1026bcae8c5aad0f239d98d3b7f', pr_url=None, repo_url=RepoUrl('https://huggingface.co/krishmurjani/finetuned-llama', endpoint='https://huggingface.co', repo_type='model', repo_id='krishmurjani/finetuned-llama'), pr_revision=None, pr_num=None)